# Problema das N Rainhas

O problema das N rainhas é um clássico desafio de xadrez e teoria dos algoritmos, onde o objetivo é posicionar N rainhas em um tabuleiro de xadrez N x N de forma que nenhuma rainha esteja atacando as outras. Isso significa que nenhuma rainha pode estar na mesma linha, coluna ou diagonal que qualquer outra rainha.

Para mais detalhes sobre as métricas e modelagem utilizadas veja o [Apêndice 4](../apendices/a-4.md)

#### Representação do Tabuleiro

In [1]:
class NQueenBoard:
    def __init__(self, N, init_pos = None):
        self.N = N
        self.board = [[0] * N for _ in range(N)]
        if init_pos:
            for i in range(N):
                self.board[init_pos[i]][i]=1
    
    def is_safe(self, row, col):
        # Verifica a linha à esquerda
        for i in range(col):
            if self.board[row][i] == 1:
                return False
        
        # Verifica a diagonal superior à esquerda
        for i, j in zip(range(row, -1, -1), range(col, -1, -1)):
            if self.board[i][j] == 1:
                return False
        
        # Verifica a diagonal inferior à esquerda
        for i, j in zip(range(row, self.N, 1), range(col, -1, -1)):
            if self.board[i][j] == 1:
                return False
        
        return True
    
    def print(self):
        print('+' + '---+' * self.N)
        for i in range(self.N):
            print('|' + '|'.join(' X ' if self.board[i][j] else '   ' for j in range(self.N)) + '|')
            print('+' + '---+' * self.N)

#### Solução usando Backtracking

In [2]:
class BacktrackSolver:
    def __init__(self, board):
        self.board = board
        
    def solve_n_queens_util(self, col):
        # Se todas as rainhas estão colocadas, retorna verdadeiro
        if col >= self.board.N:
            return True
        
        for i in range(self.board.N):
            if self.board.is_safe(i, col):
                # Coloca a rainha
                self.board.board[i][col] = 1
                
                # Recursão para colocar o restante das rainhas
                if self.solve_n_queens_util(col + 1):
                    return True
                
                # Se colocar a rainha em board[i][col]
                # não conduz a uma solução, então remove a rainha (backtracking)
                self.board.board[i][col] = 0
        
        # Se a rainha não pode ser colocada em nenhuma linha nesta coluna, retorna falso
        return False
    
    def solve_n_queens(self):
        if not self.solve_n_queens_util(0):
            print("Não existe solução.")
            return None
        return self.board

In [3]:
# Aplicando o uso do backtracking
board = NQueenBoard(8)
solver = BacktrackSolver(board)
result = solver.solve_n_queens()
if result:
    print("Tabuleiro com solução:")
    result.print()
else:
    print("Não foi possível encontrar solução.")

Tabuleiro com solução:
+---+---+---+---+---+---+---+---+
| X |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   | X |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   | X |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   | X |
+---+---+---+---+---+---+---+---+
|   | X |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   | X |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   | X |   |   |
+---+---+---+---+---+---+---+---+
|   |   | X |   |   |   |   |   |
+---+---+---+---+---+---+---+---+


#### Busca de Subida de Encosta

In [4]:
class HillClimbingSearchSolver:
    def __init__(self, board):
        self.board = board
            
    # Contador de ataques em todas direções
    def count_attacks_this(self, row, col):
        count = 0
        # Contagem de atacantes na linha
        for i in range(self.board.N):
            if self.board.board[row][i] == 1 and i != col:
                count+=1
        
        # Contagem de atacantes na coluna
        for i in range(self.board.N):
            if self.board.board[i][col] == 1 and i != row:
                count+=1
        
        # Contagem de atacantes na diagonal superior à esquerda
        for i, j in zip(range(row-1, -1, -1), range(col-1, -1, -1)):
            if self.board.board[i][j] == 1:
                count+=1
        # Contagem de atacantes na diagonal superior à direita
        for i, j in zip(range(row-1, -1, -1), range(col+1, self.board.N, 1)):
            if self.board.board[i][j] == 1:
                count+=1
            
        # Contagem de atacantes na diagonal inferior à esquerda
        for i, j in zip(range(row+1, self.board.N, 1), range(col-1, -1, -1)):
            if self.board.board[i][j] == 1:
                count+=1
        
        # Contagem de atacantes na diagonal inferior à direita
        for i, j in zip(range(row+1, self.board.N, 1), range(col+1, self.board.N, 1)):
            if self.board.board[i][j] == 1:
                count+=1
        
        return count
    
    def solve_n_queens(self):
        # Geração de uma solução inicial
        for col in range(self.board.N):
            self.board.board[0][col] = 1
        
        move = True
        while move:
            move = False
            for col in range(self.board.N):
                min_attacks = self.board.N  # Inicialização da variável com um valor máximo
                best_row = 0  # Inicialização da melhor linha como 0
                current_row = None
                for row in range(self.board.N):
                    if self.board.board[row][col] == 1:
                        current_row = row  # Salva a linha atual da rainha
                        self.board.board[row][col] = 0  # Remove temporariamente a rainha
                    attacks = self.count_attacks_this(row, col)  # Calcula os ataques na posição atual
                    if attacks <= min_attacks:
                        min_attacks = attacks  # Atualiza o menor número de ataques
                        best_row = row  # Atualiza a melhor linha
                self.board.board[best_row][col] = 1  # Coloca a rainha na melhor posição encontrada
                if best_row != current_row:  # Verifica se houve movimento
                    move = True  # Se houve movimento, atualiza a flag
                    
        return self.board

In [5]:
# Aplicando busca por subida de encosta
board = NQueenBoard(8)
solver = HillClimbingSearchSolver(board)
result = solver.solve_n_queens()
if result:
    print("Tabuleiro com solução:")
    result.print()
else:
    print("Não foi possível encontrar solução.")

Tabuleiro com solução:
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   | X |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   | X |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   | X |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   | X |
+---+---+---+---+---+---+---+---+
|   |   |   |   | X |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   | X |   |   |   | X |   |
+---+---+---+---+---+---+---+---+
| X |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+


#### Busca com Algoritmo Genético

In [6]:
import random

class NQueensGeneticSolver:
    def __init__(self, N, population_size=10, mutation_rate=0.1, generations=1000):
        # Inicialização do solver com os parâmetros do algoritmo genético
        self.population_size = population_size  # Tamanho da população
        self.N = N  # Tamanho do tabuleiro (número de rainhas)
        self.mutation_rate = mutation_rate  # Taxa de mutação
        self.generations = generations  # Número de gerações

    def generate_initial_population(self):
        # Geração da população inicial de indivíduos (possíveis soluções)
        return [[random.randint(0, self.N-1) for _ in range(self.N)] for _ in range(self.population_size)]

    def fitness(self, individual):
        # Função de avaliação da aptidão de um indivíduo (solução)
        # Calcula o número de ataques entre as rainhas no tabuleiro
        attacks = 0
        for i in range(len(individual)):
            for j in range(i + 1, len(individual)):
                if individual[i] == individual[j] or abs(i - j) == abs(individual[i] - individual[j]):
                    attacks += 1
        # Quanto menor o número de ataques, melhor a aptidão (fitness)
        return self.N - attacks

    def select(self, population):
        # Seleção dos indivíduos mais aptos (com melhor fitness)
        # Seleciona os dois melhores indivíduos da população atual
        population_fitness = [(individual, self.fitness(individual)) for individual in population]
        population_fitness.sort(key=lambda x: x[1], reverse=True)  # Ordena por fitness (decrescente)
        return [individual for individual, fitness in population_fitness[:2]]  # Retorna os dois melhores indivíduos

    def mutate(self, individual):
        # Mutação de um indivíduo com base na taxa de mutação
        # Altera aleatoriamente uma posição do tabuleiro
        if random.random() < self.mutation_rate:
            idx = random.randint(0, self.N-1)  # Índice da posição a ser alterada
            individual[idx] = random.randint(0, self.N-1)  # Nova posição aleatória
        return individual

    def crossover(self, parent1, parent2):
        # Cruzamento (crossover) de dois indivíduos para gerar descendentes (filhos)
        # Seleciona um ponto de corte aleatório e cria dois descendentes combinando os pais
        idx = random.randint(1, self.N-1)  # Ponto de corte
        return parent1[:idx] + parent2[idx:], parent2[:idx] + parent1[idx:]  # Descendentes gerados

    def solve(self):
        # Resolução do problema usando o algoritmo genético
        population = self.generate_initial_population()  # Geração da população inicial
        for generation in range(self.generations):
            new_population = []  # Nova população (geração)
            while len(new_population) < self.population_size:
                parents = self.select(population)  # Seleção dos pais
                offspring1, offspring2 = self.crossover(*parents)  # Cruzamento (crossover)
                new_population.append(self.mutate(offspring1))  # Mutação e adição do primeiro descendente
                if len(new_population) < self.population_size:
                    new_population.append(self.mutate(offspring2))  # Mutação e adição do segundo descendente
            population = new_population  # Atualização da população para a próxima geração
            best_solution = max(population, key=self.fitness)  # Melhor solução da geração atual
            if self.fitness(best_solution) == self.N:  # Se a solução é ótima (sem ataques)
                print(f"Solução encontrada na geração {generation}: {best_solution}")  # Exibe a solução encontrada
                return best_solution  # Retorna a solução
        return None  # Se não encontrou uma solução ótima em todas as gerações, retorna None


In [7]:
# Aplicando busca por algoritmo genético
N=8
solver = NQueensGeneticSolver(N)
solution = solver.solve()
if solution:
    board = NQueenBoard(N, init_pos=solution)
    print("Tabuleiro com solução:")
    board.print()
else:
    print("Não foi possível encontrar solução.")

Não foi possível encontrar solução.


#### Busca com CSP

In [10]:
# versão incompleta, ainda vou corrigir!
class NQueenCSPSolver:
    def __init__(self, N):
        self.N = N
        self.domains = {i: set(range(N)) for i in range(N)}  # Domínio de cada variável
        self.variables = list(range(N))                      # Variáveis do problema
        self.constraints = self.generate_constraints()       # Geração das restrições do problema

    def generate_constraints(self):
        constraints = []
        # Gera as restrições de que nenhuma rainha pode atacar outra
        for i in range(self.N):
            for j in range(i + 1, self.N):
                constraints.append(((i, j), self.not_attacking))  # Armazena a restrição e a função de restrição
        return constraints

    def not_attacking(self, pos1, pos2):
        # Função que verifica se duas rainhas não estão se atacando
        row1, col1 = pos1
        row2, col2 = pos2
        # Duas rainhas não se atacam se não estão na mesma linha, coluna ou diagonal
        return row1 != row2 and col1 != col2 and abs(row1 - row2) != abs(col1 - col2)

    def is_consistent(self, var, value, assignment):
        # Verifica se a atribuição é consistente com as restrições
        for neighbor, constraint_func in self.constraints:
            if var == neighbor[1] and neighbor[0] in assignment:
                if not constraint_func((assignment[neighbor[0]], var), (value, var)):
                    return False
        return True

    def select_unassigned_variable(self, assignment):
        # Seleciona uma variável não atribuída
        for var in self.variables:
            if var not in assignment:
                return var
        return None

    def backtrack(self, assignment):
        # Algoritmo de backtrack para encontrar a solução
        if len(assignment) == self.N:  # Se todas as variáveis foram atribuídas
            return assignment
        var = self.select_unassigned_variable(assignment)  # Seleciona uma variável não atribuída
        for value in self.domains[var]:  # Para cada valor no domínio da variável
            if self.is_consistent(var, value, assignment):  # Se a atribuição é consistente com as restrições
                assignment[var] = value  # Atribui o valor à variável
                result = self.backtrack(assignment)  # Realiza a busca recursiva
                if result is not None:  # Se encontrou uma solução
                    return result
                del assignment[var]  # Se não encontrou, desfaz a atribuição
        return None

    def solve_n_queens(self):
        # Função principal para resolver o problema das N Rainhas
        assignment = {}  # Dicionário para armazenar as atribuições
        result = self.backtrack(assignment)  # Chama o algoritmo de backtrack
        if result is None:  # Se não encontrou uma solução
            return None
        list_result = [0] * self.N
        for var, value in result.items():  # Para cada variável atribuída
            list_result[var] = value  # Coloca uma rainha na posição correspondente
        return list_result


# Testando o solver
N = 4  # Tamanho do tabuleiro
solver = NQueenCSPSolver(N)  # Inicializa o solver
solution = solver.solve_n_queens()  # Resolve o problema

if solution:
    board = NQueenBoard(N, init_pos=solution)  # Cria um objeto do tabuleiro com a solução encontrada
    print("Tabuleiro com solução:")
    board.print()  # Imprime o tabuleiro com a solução
else:
    print("Não foi possível encontrar solução.")


Não foi possível encontrar solução.


#### CSP com library constraint

A seguir apresentamos a solução do problema das N rainhas utilizando a biblioteca Python de CSP.

Para utilizar é necessário instalar a biblioteca:
```shell
pip install python-constraint
```

In [9]:
from constraint import Problem

def n_queens_solver(n):
    # Cria o problema CSP
    problem = Problem()

    # Adiciona as variáveis (cada variável representa uma coluna e seu valor, a linha ocupada pela rainha)
    for i in range(n):
        problem.addVariable(i, range(n))

    # Adiciona as restrições para garantir que as rainhas não ataquem umas às outras
    for i in range(n):
        for j in range(i + 1, n):
            # Duas rainhas não podem estar na mesma linha
            problem.addConstraint(lambda x, y: x != y, (i, j))

            # Duas rainhas não podem estar na mesma diagonal
            problem.addConstraint(lambda x, y, i=i, j=j: abs(i - j) != abs(x - y), (i, j))

    # Encontra e retorna uma solução
    return problem.getSolutions()

n = 6
solutions = n_queens_solver(n)
print(len(solutions))
list_result = [0] * n
for solution in solutions:
    for var, value in solution.items():  # Para cada variável atribuída
        list_result[var] = value  # Coloca uma rainha na posição correspondente
    print(list_result)


4
[4, 2, 0, 5, 3, 1]
[3, 0, 4, 1, 5, 2]
[2, 5, 1, 4, 0, 3]
[1, 3, 5, 0, 2, 4]
